In [6]:
import pandas as pd
import datetime
import ast

news = pd.read_csv("news.csv")
price = pd.read_csv("price.csv")

In [7]:
result = open("result.txt", "r")
result_lst = ast.literal_eval(result.readline())
result_date = {}

for item in result_lst:
    result_date[item["ticker"]] = item["newsStartDate"]
result_date

{'ABBV': '2016-08-01',
 'ABT': '2016-07-28',
 'ADM': '2016-06-27',
 'ADP': '2016-08-02',
 'AFL': '2016-07-25',
 'ALB': '2016-07-13',
 'AMCR': '2020-01-14',
 'AOS': '2016-07-26',
 'APD': '2016-07-29',
 'ATO': '2016-10-28',
 'BDX': '2016-07-15',
 'BEN': '2016-07-28',
 'BF.B': '2018-01-04',
 'CAH': '2016-07-13',
 'CAT': '2016-07-27',
 'CB': '2016-07-26',
 'CINF': '2016-07-18',
 'CL': '2016-08-01',
 'CLX': '2016-07-01',
 'CTAS': '2016-07-19',
 'CVX': '2016-07-13',
 'DOV': '2016-07-19',
 'ECL': '2016-06-01',
 'ED': '2016-07-25',
 'EMR': '2016-07-21',
 'ESS': '2016-09-28',
 'EXPD': '2016-10-14',
 'FRT': '2016-07-13',
 'GD': '2016-07-13',
 'GPC': '2016-08-26',
 'GWW': '2016-06-29',
 'HRL': '2016-05-18',
 'ITW': '2016-07-26',
 'JNJ': '2016-05-17',
 'KMB': '2016-07-20',
 'KO': '2016-07-07',
 'LEG': '2016-07-08',
 'LIN': '2019-04-30',
 'LOW': '2016-08-04',
 'MCD': '2016-07-07',
 'MDT': '2016-07-05',
 'MKC': '2016-06-14',
 'MMM': '2016-07-26',
 'NUE': '2016-07-01',
 'O': '2016-08-02',
 'PBCT': '2

In [8]:
def merge_dataframe(news_csv, price_csv, cut_date):
    
    news_data = news_csv.loc[:, ['ticker', 'date', 'title','short_description']]
    news_data = news_data.drop_duplicates()
    # aggregate all title and short description info on the same date and merge the datadrame
    news_data = news_data.groupby(['ticker','date'])[['title','short_description']].agg(list)
    
    price_data = price_csv.loc[:, ['ticker', 'date', 'close']]
    price_data = price_data.drop_duplicates()
    df = pd.merge(news_data, price_data, how="outer", on=['ticker', 'date'])

    df = df.sort_values(['ticker', 'date'], ascending=[True, True])
    df = df.fillna(method='ffill')
    
    for name in df['ticker'].unique().tolist():
        if name in cut_date:
            df = df.drop(df[(df['ticker'] == name) & (df['date'] < cut_date[name] )].index)
    df.reset_index(inplace=True)
    del df['index']
    return df
    

In [9]:
df = merge_dataframe(news,price,result_date)
df

,ticker,date,title,short_description,close
0,ABBV,2016-08-01,"[Company News for August 01, 2016]",[• Shares of Xerox Corp ( XRX ) gained 3.9%...,66.94
1,ABBV,2016-08-02,"[Company News for August 01, 2016]",[• Shares of Xerox Corp ( XRX ) gained 3.9%...,66.78
2,ABBV,2016-08-03,"[Company News for August 01, 2016]",[• Shares of Xerox Corp ( XRX ) gained 3.9%...,66.57
3,ABBV,2016-08-04,"[Healthcare Dividend Growth Stocks: Part 1, ET...","[Healthcare Dividend Growth Stocks: Part 1, ET...",66.10
4,ABBV,2016-08-05,[Elliott Associates sues AbbVie over failed bi...,[Elliott Associates sues AbbVie over failed bi...,66.54
...,...,...,...,...,...
53107,XOM,2020-04-01,[Trump warns of ‘heavy price’ if Iran attacks ...,[President Donald Trump warned Iran on Wednesd...,37.53
53108,XOM,2020-04-02,[Oil in Sudden U-Turn After Trump Touts Produc...,[Can Donald Trump achieve what OPEC itself cou...,40.40
53109,XOM,2020-04-03,[Trump Faces Bitterly Divided Oil Producers in...,"[(Bloomberg) -- In Saudi Arabia, there is one ...",39.21
53110,XOM,2020-04-04,[Exclusive: Honeywell Pressures Suppliers To C...,"[Honeywell, the big industrial and aerospace f...",39.21


In [10]:
def set_label(day, df):
    
    # create shiftday price column 
    df["shiftday"] = df["close"].shift(-day, axis = 0)
    df = df.fillna(list(df["close"])[-1])
    
    # create price difference column
    df["price_diff"] = df["shiftday"] - df["close"]
    
    # create price difference column
    df["label"] = df["price_diff"].apply(lambda x: 1 if x > 0 else 0)
    
    return df

In [11]:
label_df = set_label(5, df)
label_df

,ticker,date,title,short_description,close,shiftday,price_diff,label
0,ABBV,2016-08-01,"[Company News for August 01, 2016]",[• Shares of Xerox Corp ( XRX ) gained 3.9%...,66.94,66.54,-0.40,0
1,ABBV,2016-08-02,"[Company News for August 01, 2016]",[• Shares of Xerox Corp ( XRX ) gained 3.9%...,66.78,66.34,-0.44,0
2,ABBV,2016-08-03,"[Company News for August 01, 2016]",[• Shares of Xerox Corp ( XRX ) gained 3.9%...,66.57,66.76,0.19,1
3,ABBV,2016-08-04,"[Healthcare Dividend Growth Stocks: Part 1, ET...","[Healthcare Dividend Growth Stocks: Part 1, ET...",66.10,66.43,0.33,1
4,ABBV,2016-08-05,[Elliott Associates sues AbbVie over failed bi...,[Elliott Associates sues AbbVie over failed bi...,66.54,66.95,0.41,1
...,...,...,...,...,...,...,...,...
53107,XOM,2020-04-01,[Trump warns of ‘heavy price’ if Iran attacks ...,[President Donald Trump warned Iran on Wednesd...,37.53,39.21,1.68,1
53108,XOM,2020-04-02,[Oil in Sudden U-Turn After Trump Touts Produc...,[Can Donald Trump achieve what OPEC itself cou...,40.40,39.21,-1.19,0
53109,XOM,2020-04-03,[Trump Faces Bitterly Divided Oil Producers in...,"[(Bloomberg) -- In Saudi Arabia, there is one ...",39.21,39.21,0.00,0
53110,XOM,2020-04-04,[Exclusive: Honeywell Pressures Suppliers To C...,"[Honeywell, the big industrial and aerospace f...",39.21,39.21,0.00,0


In [12]:
result_df = label_df.drop("price_diff", axis =1)
result_df

,ticker,date,title,short_description,close,shiftday,label
0,ABBV,2016-08-01,"[Company News for August 01, 2016]",[• Shares of Xerox Corp ( XRX ) gained 3.9%...,66.94,66.54,0
1,ABBV,2016-08-02,"[Company News for August 01, 2016]",[• Shares of Xerox Corp ( XRX ) gained 3.9%...,66.78,66.34,0
2,ABBV,2016-08-03,"[Company News for August 01, 2016]",[• Shares of Xerox Corp ( XRX ) gained 3.9%...,66.57,66.76,1
3,ABBV,2016-08-04,"[Healthcare Dividend Growth Stocks: Part 1, ET...","[Healthcare Dividend Growth Stocks: Part 1, ET...",66.10,66.43,1
4,ABBV,2016-08-05,[Elliott Associates sues AbbVie over failed bi...,[Elliott Associates sues AbbVie over failed bi...,66.54,66.95,1
...,...,...,...,...,...,...,...
53107,XOM,2020-04-01,[Trump warns of ‘heavy price’ if Iran attacks ...,[President Donald Trump warned Iran on Wednesd...,37.53,39.21,1
53108,XOM,2020-04-02,[Oil in Sudden U-Turn After Trump Touts Produc...,[Can Donald Trump achieve what OPEC itself cou...,40.40,39.21,0
53109,XOM,2020-04-03,[Trump Faces Bitterly Divided Oil Producers in...,"[(Bloomberg) -- In Saudi Arabia, there is one ...",39.21,39.21,0
53110,XOM,2020-04-04,[Exclusive: Honeywell Pressures Suppliers To C...,"[Honeywell, the big industrial and aerospace f...",39.21,39.21,0


In [13]:
result_df['short_description'] = result_df['short_description'].apply(lambda x: list(set(x)))
result_df['short_description'] = [','.join(map(str, i)) for i in df['short_description']]
result_df

,ticker,date,title,short_description,close,shiftday,label
0,ABBV,2016-08-01,"[Company News for August 01, 2016]",• Shares of Xerox Corp ( XRX ) gained 3.9% ...,66.94,66.54,0
1,ABBV,2016-08-02,"[Company News for August 01, 2016]",• Shares of Xerox Corp ( XRX ) gained 3.9% ...,66.78,66.34,0
2,ABBV,2016-08-03,"[Company News for August 01, 2016]",• Shares of Xerox Corp ( XRX ) gained 3.9% ...,66.57,66.76,1
3,ABBV,2016-08-04,"[Healthcare Dividend Growth Stocks: Part 1, ET...","Healthcare Dividend Growth Stocks: Part 1,ETF’...",66.10,66.43,1
4,ABBV,2016-08-05,[Elliott Associates sues AbbVie over failed bi...,Elliott Associates sues AbbVie over failed bid...,66.54,66.95,1
...,...,...,...,...,...,...,...
53107,XOM,2020-04-01,[Trump warns of ‘heavy price’ if Iran attacks ...,President Donald Trump warned Iran on Wednesda...,37.53,39.21,1
53108,XOM,2020-04-02,[Oil in Sudden U-Turn After Trump Touts Produc...,Can Donald Trump achieve what OPEC itself coul...,40.40,39.21,0
53109,XOM,2020-04-03,[Trump Faces Bitterly Divided Oil Producers in...,"(Bloomberg) -- In Saudi Arabia, there is one o...",39.21,39.21,0
53110,XOM,2020-04-04,[Exclusive: Honeywell Pressures Suppliers To C...,"Honeywell, the big industrial and aerospace fi...",39.21,39.21,0


In [15]:
from nltk.book import *

from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [21]:
def sentiment(df):
    sid = SentimentIntensityAnalyzer()
    df['sentiment'] = df['short_description'].apply(lambda x:sid.polarity_scores(str(x)))
    df['pos'] = df['sentiment'].apply(lambda x: x['pos'])
    df['neu'] = df['sentiment'].apply(lambda x: x['neu'])
    df['neg'] = df['sentiment'].apply(lambda x: x['neg'])
    df['compound'] = df['sentiment'].apply(lambda x: x['compound'])
    
    df = df[['date','short_description','label','pos','neu','neg','compound']]
    return df

In [22]:
sentiment(result_df)

,date,short_description,label,pos,neu,neg,compound
0,2016-08-01,• Shares of Xerox Corp ( XRX ) gained 3.9% ...,0,0.286,0.638,0.077,0.7351
1,2016-08-02,• Shares of Xerox Corp ( XRX ) gained 3.9% ...,0,0.286,0.638,0.077,0.7351
2,2016-08-03,• Shares of Xerox Corp ( XRX ) gained 3.9% ...,1,0.286,0.638,0.077,0.7351
3,2016-08-04,"Healthcare Dividend Growth Stocks: Part 1,ETF’...",1,0.061,0.939,0.000,0.3818
4,2016-08-05,Elliott Associates sues AbbVie over failed bid...,1,0.053,0.828,0.119,-0.7964
...,...,...,...,...,...,...,...
53107,2020-04-01,President Donald Trump warned Iran on Wednesda...,1,0.070,0.818,0.112,-0.9919
53108,2020-04-02,Can Donald Trump achieve what OPEC itself coul...,0,0.086,0.868,0.046,0.9931
53109,2020-04-03,"(Bloomberg) -- In Saudi Arabia, there is one o...",0,0.134,0.803,0.063,0.9998
53110,2020-04-04,"Honeywell, the big industrial and aerospace fi...",0,0.038,0.884,0.078,-0.2263
